In [5]:
from utils.connect import *
from utils.kit import *
from utils.search import * 

def get_connection():
    return MongoClient("mongodb://127.0.0.1:27017")

client = get_connection()
db = client.ZMS

ModuleNotFoundError: No module named 'utils.search'

NameError: name 'insert_with_drop_dubs' is not defined

In [2]:


def eval_sample(sample, pattern, keys, nr_of_errors_possible=0):
    """
    Function to evalute two samples
    keys - list with common allels
    """

    nr_of_errors = 0
    if len(keys) == 0:
        return 99
    for i in keys:
        s_in_p = len([j for j in sample[i] if j in pattern[i]])
        p_in_s = len([j for j in pattern[i] if j in sample[i]])
        if s_in_p ==2 and p_in_s == 1 : 
            nr_of_errors +=1
        if s_in_p == 1: 
            nr_of_errors +=1 
        if s_in_p == 0: 
            nr_of_errors +=2

        if nr_of_errors > nr_of_errors_possible:
            return 99
    return nr_of_errors

In [7]:

def mongoDB_search(pattern: dict, nr_of_errors_possible=0):
    """
    Function to search database with posible nr_of_errors_possible
    pattern - dicionary {allel : values} only with no NA values
    """

    # conection chec
    if db["profile"].count() == 0:
        raise Exception("No conection to database")

    # search
    id_to_return = []
    nr_of_errors_to_return =[]
    pattern_keys = list(pattern.keys())
    # Do zmany w całym pakiecie 
    for cur_sample in db["profile"].find({}):

        sample_keys = list(cur_sample['allels'])

        keys_to_check = list(set(pattern_keys) & set(sample_keys))
        curr_nr_of_errors = eval_sample(cur_sample['allels'], pattern, keys_to_check, nr_of_errors_possible)      
        if  curr_nr_of_errors <= nr_of_errors_possible:
            id_to_return.append(cur_sample)
            nr_of_errors_to_return.append(curr_nr_of_errors)

    return id_to_return , nr_of_errors_to_return


In [16]:
test = {"CSF1PO": [11.0, 12.0], "TH01": [8, 9.3], "vWA": [16.0, 16.0], "TPOX": [8.0, 8.0]}

res,errors = mongoDB_search(test,0)
print(res)

[{'_id': ObjectId('6177148d4ca46a197426a507'), 'allels': {'CSF1PO': [11.0, 12.0], 'TH01': [8, 9.3], 'vWA': [16.0, 16.0], 'TPOX ': [8.0, 8.0]}, 'opinion': '14/98', 'sample': 'NN 122'}, {'_id': ObjectId('6177148d4ca46a197426a508'), 'allels': {'CSF1PO': [11.0, 12.0], 'TH01': [8, 9.3], 'vWA': [16.0, 16.0], 'TPOX ': [8.0, 8.0]}, 'opinion': '14/98', 'sample': 'Sebastian Rusinowski'}, {'_id': ObjectId('6177148d4ca46a197426a509'), 'allels': {'CSF1PO': [11.0, 12.0], 'TH01': [8, 9.3], 'vWA': [16.0, 16.0], 'TPOX ': [8.0, 8.0]}, 'opinion': '14/98', 'sample': 'Elżbieta Rusinowskia'}, {'_id': ObjectId('6177148d4ca46a197426a50a'), 'allels': {'CSF1PO': [11.0, 12.0], 'TH01': [8, 9.3], 'vWA': [16.0, 16.0], 'TPOX ': [8.0, 8.0]}, 'opinion': '42/99', 'sample': 1.81}, {'_id': ObjectId('6177148d4ca46a197426a50b'), 'allels': {'CSF1PO': [11.0, 12.0], 'TH01': [8, 9.3], 'vWA': [16.0, 16.0], 'TPOX ': [8.0, 8.0]}, 'opinion': '42/99', 'sample': 2}, {'_id': ObjectId('6177148d4ca46a197426a50c'), 'allels': {'CSF1PO': 

C:\Users\janbo\AppData\Local\Temp/ipykernel_4344/2800854640.py:8: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  if db["profile"].count() == 0:


In [17]:
for cur_sample in db["profile"].find({}):
    print(cur_sample['allels'])

{'AMEL': ['X', 'Y'], 'CSF1PO': ['12', '12'], 'D10S1248': ['13', '15'], 'D12S391': ['18', '23'], 'D13S317': ['9', '11'], 'D16S539': ['9', '11'], 'D18S51': ['16', '18'], 'D19S433': ['13', '14'], 'D1S1656': ['12', '13'], 'D21S11': ['29', '31.2'], 'D22S1045': ['16', '16'], 'D2S1338': ['22', '25'], 'D2S441': ['10', '14'], 'D3S1358': ['17', '18'], 'D5S818': ['12', '12'], 'D7S820': ['8', '11'], 'D8S1179': ['14', '15'], 'DYS391': ['10', '10'], 'DYS570': ['17', '17'], 'DYS576': ['18', '18'], 'FGA': ['20', '23'], 'Penta D': ['12', '13'], 'Penta E': ['7', '14'], 'SE33': ['15', '16'], 'TH01': ['6', '9.3'], 'TPOX': ['11', '11'], 'vWA': ['16', '19']}
{'AMEL': ['X', 'Y'], 'CSF1PO': ['12', '12'], 'D10S1248': ['13', '15'], 'D12S391': ['18', '23'], 'D13S317': ['9', '11'], 'D16S539': ['9', '11'], 'D18S51': ['16', '18'], 'D19S433': ['13', '14'], 'D1S1656': ['12', '13'], 'D21S11': ['29', '31.2'], 'D22S1045': ['16', '16'], 'D2S1338': ['22', '25'], 'D2S441': ['10', '14'], 'D3S1358': ['17', '18'], 'D5S818': [